Just adding a bit of $\LaTeX$ - seems needed to make it render below...

In [73]:
import ipywidgets as widgets
from IPython.display import display
import collision_driver_models as cdm
import importlib
importlib.reload(cdm)

NO_PRESET_STRING = '<You can choose a parameter preset here>'

class ParametersWidget:
    def __init__(self, parameterizable, is_togglable = False, toggled_on = True):
        self.parameterizable = parameterizable
        self.is_togglable = is_togglable
        ## header
        #header_widget = widgets.HTML(value = '<b>' + parameterizable.name + '</b>')
        #display(header_widget)
        # parameters
        accordion_widgets = []
        self.param_widgets = {}
        for i, param_def in enumerate(parameterizable.param_defs):
            param_val = parameterizable.param_vals[param_def.short_name]
            if param_def.type is cdm.ParameterType.BOOLEAN:
                param_widget = widgets.Checkbox(description = param_def.display_name, value = param_val)
                param_widget.style.description_width = '0'
                #display(param_widget)
                accordion_widgets.append(param_widget)
            elif param_def.type is cdm.ParameterType.COLOR:
                param_widget = widgets.ColorPicker(description = param_def.display_name, value = param_val)
                #display(param_widget)
                accordion_widgets.append(param_widget)
            else:
                min_val = param_def.range[0]
                max_val = param_def.range[1]
                if param_def.type is cdm.ParameterType.FLOAT:
                    param_widget = widgets.BoundedFloatText(min = min_val, max = max_val, value = param_val)
                else:
                    param_widget = widgets.BoundedIntText(min = min_val, max = max_val, value = param_val)
                hbox = widgets.HBox([widgets.Label(param_def.display_name + ' (' + param_def.unit + ')'), 
                                     param_widget])
                #display(hbox)
                accordion_widgets.append(hbox)
            self.param_widgets[param_def.short_name] = param_widget
        # presets
        if len(parameterizable.presets) > 0:
            drop_options = []
            drop_options.append(NO_PRESET_STRING)
            for preset in parameterizable.presets:
                drop_options.append(preset.display_name)
            self.preset_widget = widgets.Dropdown(options = drop_options)
            #display(self.preset_widget)
            accordion_widgets.append(self.preset_widget)
            self.preset_widget.observe(self.on_preset_selected)
        # accordion
        self.accordion_widget = widgets.Accordion(children = [widgets.VBox(accordion_widgets)])
        self.accordion_widget.set_title(0, parameterizable.name)
        # toggle?
        if is_togglable:
            self.toggle = widgets.Checkbox(value = toggled_on, description = '')
            self.toggle.style.description_width = '0'
            hbox = widgets.HBox([self.toggle, self.accordion_widget])
            self.toggle.layout.width = '4%'
            self.toggle.observe(self.on_toggle, names = 'value')
            self.accordion_widget.layout.width = '96%'
            self.accordion_widget.selected_index = None
            display(hbox)
        else:
            display(self.accordion_widget)
    
    def set_parameterizable_from_widget(self):
        for param_def in self.parameterizable.param_defs:
            self.parameterizable.param_vals[param_def.short_name] = (
                self.param_widgets[param_def.short_name].value)
            
    def set_widget_from_parameterizable(self):
        for param_def in self.parameterizable.param_defs:
            self.param_widgets[param_def.short_name].value = (
                self.parameterizable.param_vals[param_def.short_name])
    
    def on_preset_selected(self, change):
        if not (self.preset_widget.value == NO_PRESET_STRING):
            # make sure parameterizable is up to date with widget 
            # (since not all parameter are necessarily set by a preset)
            self.set_parameterizable_from_widget()
            # apply the preset
            self.parameterizable.choose_preset(self.preset_widget.value)
            # pull back parameter values to widget
            self.set_widget_from_parameterizable()
            # set the preset widgetback to the no-choice option
            self.preset_widget.value = NO_PRESET_STRING
            
    def on_toggle(self, change):
        # collapse the parameter accordion if parameters widget not toggled on
        if not self.toggle.value:
            self.accordion_widget.selected_index = None
            
            

scenario = cdm.Scenario()
sim_engine = cdm.SimulationEngine(scenario)
sim_engine.param_vals['end_time'] = 20
sim_engine.param_vals['n_simulations'] = 100
sim_engine.add_driver_model(cdm.FixedLDTModel())
sim_engine.add_driver_model(cdm.MaddoxAndKiefer2012Model())
sim_engine.add_driver_model(cdm.MarkkulaEtAl2016Model())


params_widgets = []
driver_model_widgets = []
params_widgets.append(ParametersWidget(scenario, is_togglable = False))
for driver_model in sim_engine.driver_models:
    driver_model_widget = ParametersWidget(driver_model, is_togglable = True)
    params_widgets.append(driver_model_widget)
    driver_model_widgets.append(driver_model_widget)
params_widgets.append(ParametersWidget(sim_engine, is_togglable = False))

output = widgets.Output()

def on_run_btn_clicked(b):
    # set parameter values
    for params_widget in params_widgets:
        params_widget.set_parameterizable_from_widget()
    # specify driver models to include
    sim_engine.clear_driver_models()
    for driver_model_widget in driver_model_widgets:
        if driver_model_widget.toggle.value:
            sim_engine.add_driver_model(driver_model_widget.parameterizable)
    # run simulations 
    run_button.description = 'Running/plotting...'
    run_button.disabled = True
    output.clear_output()
    with output:
        try:
            # run simulations
            sim_engine.simulate_driver_models()
            # plot
            sim_engine.plot()
        except Exception as err:
            print('Failed with error message: ' + format(err))
    run_button.description = 'Run'
    run_button.disabled = False
        

run_button = widgets.Button(description = 'Run')
run_button.on_click(on_run_btn_clicked)
display(run_button)
display(output)


Accordion(children=(VBox(children=(HBox(children=(Label(value='Initial speed of ego vehicle (m/s)'), BoundedFl…

Accordion(children=(VBox(children=(HBox(children=(Label(value='Simulation/plotting end time (s)'), BoundedFloa…

Button(description='Run', style=ButtonStyle())

Output()